In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

# Gemini libraries
from google import genai
from google.genai import types

In [2]:
# To manage environment variables
from google.colab import userdata

**DATASET**

*NER tags*:

- **ORGANIZATION** such as *Georgia-Pacific Corp.*, *WHO
- **PERSON** such as *Eddy Bonte*, *President Obama*.
- **LOCATION** such as *Murray River*, *Mount Everest*.
- **DATE** - such as *June*, *2008-06-29*.
- **TIME** - such as *two fifty a m*, *1:30 p.m.*
- **MONEY** such as *175 million Canadian Dollars*, *GBP 10.40*.
- **PERCENT** such as *twenty pct*, *18.75 %*
- **FACILITY** such as *Washington Monument*, *Stonehenge*.
- **GPE** such as *South East Asia*, *Midlothian*.

In [3]:
# Use your personal account!

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
namefile = '/content/drive/MyDrive/Colab Notebooks/NLP/ner_dataset.csv'

dataset = pd.read_csv(namefile, encoding="utf-8")

print("Dataset head:")
print(dataset.head)
print("\nDataset shape: ", dataset.shape)
print("\nDataset info:")
print(dataset.info())

Dataset head:
<bound method NDFrame.head of                                                     text  \
0      Thousands of demonstrators have marched throug...   
1      Iranian officials say they expect to get acces...   
2      Helicopter gunships Saturday pounded militant ...   
3      They left after a tense hour-long standoff wit...   
4      U.N. relief coordinator Jan Egeland said Sunda...   
...                                                  ...   
47954  Opposition leader Mir Hossein Mousavi has said...   
47955  On Thursday , Iranian state media published a ...   
47956  Following Iran 's disputed June 12 elections ,...   
47957  Since then , authorities have held public tria...   
47958  The United Nations is praising the use of mili...   

                                                  labels  
0      O O O O O O B-geo O O O O O B-geo O O O O O B-...  
1      B-gpe O O O O O O O O O O O O O O B-tim O O O ...  
2      O O B-tim O O O O O B-geo O O O O O B-org O O ...  

In [22]:
# Load the dataset

total_rows = len(pd.read_csv(namefile, encoding="utf-8"))
pbar = tqdm(total=total_rows)

# Create a new np array to store the extracted sentences
dataset_sentences_array = []
dataset_labels_array = []

chunksize = 10000

# Open file
for chunk in pd.read_csv(namefile, sep=',', encoding="utf-8", chunksize=chunksize):
    # Manage NaN values
    chunk = chunk.fillna('')

    for index, row in chunk.iterrows():

        # Save string and true label
        dataset_sentences_array.append(row['text'])
        dataset_labels_array.append(row['labels'])

    pbar.update(chunksize)

pbar.close()

# Print out some info

print("\nAn element from the sentence array: ", dataset_sentences_array[10])
print("An element from the true label array: ", dataset_labels_array[10])

print("Len sentence array: ", len(dataset_sentences_array))
print("Len true label array: ", len(dataset_labels_array))


50000it [00:02, 18066.62it/s]


An element from the sentence array:  In Beirut , a string of officials voiced their anger , while at the United Nations summit in New York , Prime Minister Fouad Siniora said the Lebanese people are resolute in preventing such attempts from destroying their spirit .
An element from the true label array:  O B-geo O O O O O O O O O O O O B-org I-org O O B-geo I-geo O B-per O B-per I-per O O B-gpe O O O O O O O O O O O O
Len sentence array:  47959
Len true label array:  47959


In [23]:
import random

# Random seed for reproducibility (seed = 77)
random.seed(77)

# Shuffle array with random seed
random.shuffle(dataset_sentences_array)
random.shuffle(dataset_labels_array)

print("An element from the sentence array: ", dataset_sentences_array[10])
print("An element from the true label array: ", dataset_labels_array[10])

# Slicing 150
dataset_sentences_array = dataset_sentences_array[:150]
dataset_labels_array = dataset_labels_array[:150]

print("Len sentence array: ", len(dataset_sentences_array))
print("Len true label array: ", len(dataset_labels_array))

An element from the sentence array:  South Korean authorities have destroyed thousands of ducks at four southern farms , after discovering what they call a " low pathogenic " strain of the bird flu virus .
An element from the true label array:  O B-org I-org I-org O O O O O O O O O
Len sentence array:  150
Len true label array:  150


**Gemini** for NER *(Name Entity Recognition)* task

In [ ]:
# Get the API key from the environment of Google Colab (aka Secrets)

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
response = ""
for sentence in dataset_sentences_array:

  #TODO:

  # Prompt for the NER task
  # prompt = ""


  # response = client.models.generate_content(
  #     model="gemini-2.0-flash",
  #     contents=prompt,
  # )

  # response = response.text

  # Analysis of the response using the true label